# Exploration des datas sur le stockage privé du projet

## Import des librairies

In [5]:
import numpy as np
import pandas as pd
import requests
import os

## Repertoire cible

In [12]:
data_path = '../../data/LAYER2/MO/train/'

## Fonctions

In [13]:
def get_total_size(directory):
    """
    Calcule la somme du poids des fichiers dans un répertoire en Mégaoctets.

    Args:
        directory (str): Chemin d'accès au répertoire.

    Returns:
        float: Somme du poids des fichiers en Mégaoctets.
    """
    total_size = 0
    for root, _, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            if os.path.isfile(filepath):
                filesize = os.path.getsize(filepath)
                total_size += filesize
    return total_size / 1024 ** 2  # Convertir en Mégaoctets

def get_champi_name(mo_db_path, names_csv_path):
    """
    Retourne le nom de la classe du champignon depuis le fichier names.csv de Mushroom Observer.
    Requiere numpy, pandas et os.

    Args:
        mo_db_path : Chemin vers le dossier contenant les classes
        names_csv_path : Chemin vers le fichier names.csv

    Returns:
        Dataframe Pandas avec IDs et noms
    """
    # Imports des sources
    data_files = os.listdir(mo_db_path)
    names = pd.read_csv(names_csv_path, delimiter='\t', index_col=0)

    # Recupération des ID des classes
    champi_classes = []
    for item in data_files:
        champi_classes.append(int(item))
    
    # Creation du DataFrame
    df = names[["text_name"]].rename(columns={'text_name': 'name'})
    df = df.loc[champi_classes]

    # Resultat
    return df

## Explorer les fichiers

In [14]:
# Liste des fichiers
data_files = os.listdir(data_path)
number_of_files = len(data_files)

print("Fichiers trouvés : ", number_of_files)

print("Liste des", number_of_files, "fichiers (", round(get_total_size(data_path), 2), "Mo ) :\n")
for item in data_files:
    print(item)

Fichiers trouvés :  23
Liste des 23 fichiers ( 441.84 Mo ) :

42
53
267
330
344
362
373
382
401
1174
1540
2749
15162
50164
63454
29997
623
47721
39842
4920
939
407
271


## Récupérer les classes (ID et noms)

In [9]:
# Récupération du fichier names.csv

def download_and_save(url, destination):
  """Télécharge un fichier depuis une URL et l'enregistre à un emplacement spécifié.

  Args:
    url: L'URL du fichier à télécharger.
    destination: Le chemin complet où enregistrer le fichier.
  """

  # Créer les répertoires s'ils n'existent pas
  os.makedirs(os.path.dirname(destination), exist_ok=True)

  # Télécharger le fichier
  response = requests.get(url, stream=True, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'})
  response.raise_for_status()  # Lever une exception si le téléchargement échoue

  with open(destination, 'wb') as f:
      for chunk in response.iter_content(chunk_size=8192):
          if chunk:  # filter out keep-alive new chunks
              f.write(chunk)

# Spécifier l'URL et le chemin de destination
url = "https://mushroomobserver.org/names.csv"
destination = "../../data/LAYER2/names.csv"

# Appeler la fonction
download_and_save(url, destination)

print(f"Le fichier a été téléchargé et enregistré à {destination}")

Le fichier a été téléchargé et enregistré à ../../data/LAYER2/names.csv


In [15]:
# Lire le fichier CSV dans un DataFrame
names = pd.read_csv("../../data/LAYER2/names.csv", delimiter='\t', index_col=0)

display(names)

,text_name,author,deprecated,correct_spelling_id,synonym_id,rank
id,,,,,,
1,Fungi,Bartl.,0,NaN,9996.0,14
2,Xylaria polymorpha group,J.D. Rogers,0,NaN,8975.0,16
3,Xylaria magnoliae,J.D. Rogers,0,NaN,NaN,4
4,Xylaria hypoxylon group,J.D. Rogers,0,NaN,3692.0,16
5,Xeromphalina,Kühner & Maire,0,NaN,6577.0,9
...,...,...,...,...,...,...
114128,"Hydnellum ""sp-IN02""",NaN,0,NaN,NaN,4
114129,"Lactifluus ""glaucescens-IN08""",S.D. Russell crypt. temp.,0,NaN,NaN,4
114130,"Clitopilus ""sp-OH01""",S.D. Russell crypt. temp.,0,NaN,NaN,4


In [16]:
champi_classes = []

for item in data_files:
    champi_classes.append(int(item))

display(champi_classes)

[42,
 53,
 267,
 330,
 344,
 362,
 373,
 382,
 401,
 1174,
 1540,
 2749,
 15162,
 50164,
 63454,
 29997,
 623,
 47721,
 39842,
 4920,
 939,
 407,
 271]

In [17]:
champid = names[["text_name"]].rename(columns={'text_name': 'name'})

champid = champid.loc[champi_classes]

display(champid)

,name
id,
42,Trametes versicolor
53,Stropharia ambigua
267,Fuligo septica
330,Cantharellus cinnabarinus
344,Boletus edulis
362,Amanita velosa
373,Amanita muscaria
382,Amanita bisporigera
401,Agaricus augustus


## Test de la fonction d'affichage des classes

In [21]:
get_champi_name(mo_db_path='../../data/LAYER2/MO/validation', names_csv_path="../../data/LAYER2/names.csv")

,name
id,
42,Trametes versicolor
53,Stropharia ambigua
267,Fuligo septica
330,Cantharellus cinnabarinus
344,Boletus edulis
362,Amanita velosa
373,Amanita muscaria
382,Amanita bisporigera
401,Agaricus augustus
